# Traceability of information

One of the main drawbacks of the data is its traceability. To ensure the quality of the data, we perform an analysis over time to look for jumps in the images (matrices), i.e. we have to make sure that there are no big jumps from one image to another so that if we run a time movie, progressive variations in the data channels are observed.

In [ ]:
import utils
import earthpy.plot as ep
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re

from datetime import datetime
from typing import Union, List, Dict, Tuple

In [ ]:
# TODO: Changes these values for parameters on a database.
BASE_DIR = '/home/aargaez/Downloads/Plant-Growth-Cycle-based-on-geospatial-data/DataExtraction/manitoba19Data'
bands =  ['B4','B3','B2','B8']

# Dict of the images.
images = utils.load_landsat_image(BASE_DIR, bands)

In [ ]:
images

In [ ]:
# Call the sort function.
sorted_keys = utils.sort_dict_by_date(images)

In [ ]:
# Iterate over the sorted keys to get the time series analysis.
for key in sorted_keys:
    # Perform operations on each band to ensure there is no jump.
    image = images[key]
    utils.display_rgb(img=image, title=key)

In [ ]:
# Iterate over the sorted keys and apply 8-bit conversion.
eight_bits_images = {key : utils.convert_to_eight_bits(images[key]) for key in sorted_keys}

In [ ]:
def time_series_calculator(
    eight_bits_dict: Union[Dict, None],
    sorted_keys: List[str]
) -> None:

    img_differences = []

    # Iterate over the sorted keys to get the time series analysis.
    for i in range(len(sorted_keys)):
        try:
            image1 = eight_bits_dict[sorted_keys[i]]
            image2 = eight_bits_dict[sorted_keys[i + 1]]
        except IndexError:
            image1 = eight_bits_dict[sorted_keys[i]]
            image2 = eight_bits_dict[sorted_keys[0]]
        
        # Element-wise sum of matrix entries.
        img_differences.append(
            cv2.subtract(image2, image1)
        )

    return img_differences

In [ ]:
time_series = time_series_calculator(eight_bits_dict=eight_bits_images, sorted_keys=sorted_keys)

In [ ]:
# Iterate over the sorted keys to get the time series analysis.
for img in time_series:
    plt.figure()
    plt.imshow(img)